# TCGA数据库预处理

我们经常从TCGA数据库下载患者生存数据进行分析，以验证基因在癌症中的重要性。然而，TCGA数据库的预处理往往是一个令人头痛的问题。在这里，我们介绍了ov中的TCGA模块，一种快速处理从TCGA数据库下载的文件格式的方法。我们需要准备3个文件作为输入:

- `gdc_sample_sheet`: (.tsv): TCGA的sample sheet按钮，我们可以从中获取tsv文件
- `gdc_download_files`（文件夹）：TCGA的下载/`Cart`车按钮，我们获取`tar.gz`包含您选择的所有文件
- `clinical_cart` (folder): TCGA的临床按钮，我们可以得到包含您所有临床文件的`tar.gz`

In [1]:
import omicverse as ov
import scanpy as sc
ov.plot_set()

/slurm/home/admin/.conda/envs/grns/lib/python3.11/site-packages/torch_geometric/typing.py:54: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: /lib64/libm.so.6: version `GLIBC_2.29' not found (required by /slurm/home/admin/.conda/envs/grns/lib/python3.11/site-packages/libpyg.so)
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/slurm/home/admin/.conda/envs/grns/lib/python3.11/site-packages/torch_geometric/typing.py:110: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /lib64/libm.so.6: version `GLIBC_2.29' not found (required by /slurm/home/admin/.conda/envs/grns/lib/python3.11/site-packages/libpyg.so)
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "



   ____            _     _    __                  
  / __ \____ ___  (_)___| |  / /__  _____________ 
 / / / / __ `__ \/ / ___/ | / / _ \/ ___/ ___/ _ \ 
/ /_/ / / / / / / / /__ | |/ /  __/ /  (__  )  __/ 
\____/_/ /_/ /_/_/\___/ |___/\___/_/  /____/\___/                                              

Version: 1.6.5, Tutorials: https://omicverse.readthedocs.io/
All dependencies are satisfied.


## TCGA计数读取
在这里，我们使用`ov.bulk.TCGA`执行您之前下载的`gdc`样表、`gdc`下载文件和临床`car`。`count`、`fpkm`和`tpm`矩阵将存储在一个data对象中


In [2]:
gdc_sample_sheep='data/TCGA_OV/gdc_sample_sheet.2024-07-05.tsv'
gdc_download_files='data/TCGA_OV/gdc_download_20240705_180129.081531'
clinical_cart='data/TCGA_OV/clinical.cart.2024-07-05'
aml_tcga=ov.bulk.pyTCGA(gdc_sample_sheep, gdc_download_files, clinical_cart)
aml_tcga.adata_init()

FileNotFoundError: [Errno 2] No such file or directory: 'data/TCGA_OV/gdc_download_20240705_180129.081531'

我们可以保存anndata对象以供下次使用

In [ ]:
aml_tcga.adata.write_h5ad('../../data/ov_tcga_raw.h5ad',compression='gzip')

In [3]:
aml_tcga = ov.read("../../data/ov_tcga_raw.h5ad")

In [4]:
aml_tcga

AnnData object with n_obs × n_vars = 429 × 60664
    layers: 'deseq_normalize', 'fpkm', 'tpm'

In [5]:
aml_tcga.adata_meta_init()

AttributeError: 'AnnData' object has no attribute 'adata_meta_init'

注意:每次读取anndata文件时，我们都需要使用三个路径初始化TCGA对象，以便后续的TCGA函数(如生存分析)可以正确使用

如果您希望创建自己的TCGA数据，我们在这里提供了示例数据供下载

TCGA OV: https://figshare.com/ndownloader/files/47461946

## Meta init
当TCGA读取基因id时，我们需要将其转换为基因名称以及添加有关患者的基本信息。因此，我们需要初始化患者的元信息。

In [6]:
aml_tcga.adata_meta_init()

AttributeError: 'AnnData' object has no attribute 'adata_meta_init'

## Survial init
我们之前为Clinical设置了路径，但实际上我们在之前的流程中并没有导入患者信息，我们只是初步确定了患者TCGA的id，所以我们开始对临床信息进行初始化

In [7]:
aml_tcga.survial_init()
aml_tcga.adata

AttributeError: 'AnnData' object has no attribute 'survial_init'

为了可视化你感兴趣的基因，我们可以用生存分析来完成它

In [8]:
aml_tcga.survival_analysis('MYC',layer='deseq_normalize',plot=True)

AttributeError: 'AnnData' object has no attribute 'survival_analysis'

如果你想计算所有基因的生存，你也可以用生存分析全部来完成它。它可以计算很多次。

In [9]:
aml_tcga.survial_analysis_all()
aml_tcga.adata

AttributeError: 'AnnData' object has no attribute 'survial_analysis_all'

不要忘记保存结果。

In [ ]:
aml_tcga.adata.write_h5ad('data/TCGA_OV/ov_tcga_survial_all.h5ad',compression='gzip')